# Рекомендация тарифов

Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. Достигнуть значения метрика качества:
$$accuracy > 0.75.$$

### Библиотеки и модули

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [8]:
import warnings
warnings.filterwarnings("ignore")

### Содержание

1. Изучение общей информации о данных
2. Разбиение данных на выборки
3. Исследование моделей
4. Проверка модели на тестовой выборке
5. Проверка модели на адекватность

## Изучение общей информации о данных

In [9]:
path = "/datasets/users_behavior.csv"

In [10]:
data = pd.read_csv(path)
data.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В наборе данных 3214 объектов. Пропусков нет. Типы данных соответствуют своим признакам. По условию задания предобработка не требуется.

## Разбиение данных на выборки

Обозначим за X - матрицу объекты-признаки, а за y - вектор значений целевой переменной. Разделим выборку на три части: обучающую, валидационную и тестовую в отношении 3:1:1.

In [12]:
X = data.loc[:, data.columns != 'is_ultra']
X.head(5)

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


In [13]:
y = data['is_ultra']
y.head(5)

0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

In [14]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, random_state=7, shuffle=True)
X_train_valid.shape, X_test.shape, y_train_valid.shape, y_test.shape

((2571, 4), (643, 4), (2571,), (643,))

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, random_state=7, test_size=0.25)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((1928, 4), (643, 4), (1928,), (643,))

## Исследование моделей

Перед нами стоит задача бинарной классификации. Задачу классификации можем решить с помощью следующих алгоритмов моделей классификации: kNN, Logistic Regression, SVM, Decision Tree, Random Forest.

Для того, чтобы подобрать наилучшие параметры для каждой из моделей, будем использовать кросс-валидацию.

### kNN

In [16]:
gs_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid={
        'n_neighbors': range(1, 50),
        'n_jobs': [-1]
    },
    cv=5,
)

In [17]:
model_knn = gs_knn.fit(X_train, y_train)
y_pred = model_knn.predict(X_valid)

In [18]:
ac_train = accuracy_score(model_knn.predict(X_train), y_train)
ac_valid = accuracy_score(y_pred, y_valid)
print(f'Лучшие параметры: {model_knn.best_params_}')
print(f'Accuracy kNN (на обуч.): {ac_train}')
print(f'Accuracy kNN (на валид.): {ac_valid}')
print(f'Отклонение метрики accuracy: {round(((ac_valid - ac_train) / ac_train * 100), 3)}%')

Лучшие параметры: {'n_jobs': -1, 'n_neighbors': 6}
Accuracy kNN (на обуч.): 0.7894190871369294
Accuracy kNN (на валид.): 0.7558320373250389
Отклонение метрики accuracy: -4.255%


### Logistic Regression

In [19]:
gs_lor = GridSearchCV(
    LogisticRegression(),
    param_grid={
        'penalty': ['l1', 'l2'],
        'n_jobs': [-1]
    },
    cv=5
)

In [20]:
model_lor = gs_lor.fit(X_train, y_train)
y_pred = model_lor.predict(X_valid)

In [21]:
ac_train = accuracy_score(model_lor.predict(X_train), y_train)
ac_valid = accuracy_score(y_pred, y_valid)
print(f'Лучшие параметры: {model_lor.best_params_}')
print(f'Accuracy Logistic Regression (на обуч.): {accuracy_score(model_lor.predict(X_train), y_train)}')
print(f'Accuracy Logistic Regression (на валид.): {accuracy_score(y_pred, y_valid)}')
print(f'Отклонение метрики accuracy: {round(((ac_valid - ac_train) / ac_train * 100), 3)}%')

Лучшие параметры: {'n_jobs': -1, 'penalty': 'l1'}
Accuracy Logistic Regression (на обуч.): 0.7375518672199171
Accuracy Logistic Regression (на валид.): 0.7573872472783826
Отклонение метрики accuracy: 2.689%


### SVM

In [22]:
svm = SVC()

In [23]:
model_svm = svm.fit(X_train, y_train)
y_pred = model_svm.predict(X_valid)

In [24]:
ac_train = accuracy_score(model_svm.predict(X_train), y_train)
ac_valid = accuracy_score(y_pred, y_valid)
print(f'Accuracy SVM (на обуч.): {accuracy_score(model_svm.predict(X_train), y_train)}')
print(f'Accuracy SVM (на валид.): {accuracy_score(y_pred, y_valid)}')
print(f'Отклонение метрики accuracy: {round(((ac_valid - ac_train) / ac_train * 100), 3)}%')

Accuracy SVM (на обуч.): 1.0
Accuracy SVM (на валид.): 0.7013996889580093
Отклонение метрики accuracy: -29.86%


### Decision Tree

In [25]:
gs_dt = GridSearchCV(
    DecisionTreeClassifier(),
    param_grid={
        'criterion': ['gini', 'entropy'],
        'max_depth': range(1, 20),
    },
    cv=5
)

In [26]:
model_dt = gs_dt.fit(X_train, y_train)
y_pred = model_dt.predict(X_valid)

In [27]:
ac_train = accuracy_score(model_dt.predict(X_train), y_train)
ac_valid = accuracy_score(y_pred, y_valid)
print(f'Лучшие параметры: {model_dt.best_params_}')
print(f'Accuracy Decision Tree (на обуч.): {accuracy_score(model_dt.predict(X_train), y_train)}')
print(f'Accuracy Decision Tree (на валид.): {accuracy_score(y_pred, y_valid)}')
print(f'Отклонение метрики accuracy: {round(((ac_valid - ac_train) / ac_train * 100), 3)}%')

Лучшие параметры: {'criterion': 'gini', 'max_depth': 7}
Accuracy Decision Tree (на обуч.): 0.8345435684647303
Accuracy Decision Tree (на валид.): 0.7884914463452566
Отклонение метрики accuracy: -5.518%


### Random Forest

In [28]:
gs_rf = GridSearchCV(
    RandomForestClassifier(),
    param_grid={
        'n_estimators': range(1, 20),
        'max_depth': range(1, 20),
        'n_jobs': [-1]
    }
)

In [29]:
model_rf = gs_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_valid)

In [30]:
ac_train = accuracy_score(model_rf.predict(X_train), y_train)
ac_valid = accuracy_score(y_pred, y_valid)
print(f'Лучшие параметры: {model_dt.best_params_}')
print(f'Accuracy Random Forest (на обуч.): {accuracy_score(model_rf.predict(X_train), y_train)}')
print(f'Accuracy Random Forest (на валид.): {accuracy_score(y_pred, y_valid)}')
print(f'Отклонение метрики accuracy: {round(((ac_valid - ac_train) / ac_train * 100), 3)}%')

Лучшие параметры: {'criterion': 'gini', 'max_depth': 7}
Accuracy Random Forest (на обуч.): 0.875
Accuracy Random Forest (на валид.): 0.8164852255054432
Отклонение метрики accuracy: -6.687%


Заметим, что метрика accuracy на обучаемой выборке больше, чем на валидацинной выборке, у моделей: kNN, SVM, Decision Tree, Random Forest. Это говорит о том, что данные модели склоны к переобучению. Больше всего к переобучению склона модель SVM (замечание, данная модель обучался без кросс-валидации. Из-за того, чтого она долго обучалася, решил обучить ее обычным способом), Decision Tree (почти такое же отклоение и у Random Forest). Модель kNN хоть и переобучается, однако не так сильно. А модель Logistic Regression точно не склона к переобучению.

Наибольше значение метрики accuracy у модели Random Forest. Берем ее за финанульную модель.

## Проверка модели на тестовой выборке

In [31]:
y_pred_test = model_rf.predict(X_test)
print(f'Accuracy Random Forest (на тесте): {accuracy_score(y_pred_test, y_test)}')

Accuracy Random Forest (на тесте): 0.8351477449455676


Данный результат вполне устраивает нас.

## Проверка модели на адекватность

In [32]:
dc = DummyClassifier()
model_dc = dc.fit(X_train, y_train)
y_pred_test = model_dc.predict(X_test)
print(f'Accuracy Dummy Classifier (на тесте): {accuracy_score(y_pred_test, y_test)}')

Accuracy Dummy Classifier (на тесте): 0.583203732503888


Обученная модель случайного леса показывает результат по метрике качества accuracy намного выше, чем "случайная" модель. Получаем, что взятая за финальную модель прошла проверку на адекватность.